# All models

Here are all the endpoints for the appliacation. To test them before implementation.

## Imports

In [ ]:
# Fix Cuda problem
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
# import torch
# print(torch.cuda.is_available())

In [ ]:
import json
import os
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# Models
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
READER_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

## *get_context()*

In [ ]:
def get_context(query, k=5):
    """ 
    Retrieves relevant context for a given query.

    Parameters:
    query (str): The input query for which context is needed.
    k (int, optional): The number of relevant context elements to retrieve (default is 5).

    Returns:
    list: A list containing relevant context elements.
    """

    # Load embeddings
    embedding_model = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        multi_process=True,
        model_kwargs={"device": "cpu"},  # replace 'cpu' by 'cuda' if you have Nvidia gpu
        encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
    )
    KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local("../outputs/rag_embeddings_thenlper_gte-small", embedding_model, allow_dangerous_deserialization=True)

    # Retrieve docs
    print(f"\nStarting retrieval for {query=}...")
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=query, k=k)

    return retrieved_docs

## *get_ai_answer(question)*
```
Improovements:
- In its v2 an agent can check and validate the quality of an answer checking the sources. He can correct the question if it's not ok.
```


In [ ]:
def get_ai_answer(question):
    """
    Generates an AI-generated response to a given question.

    Parameters:
    question (str): The input question for which an answer is needed.

    Returns:
    str: The generated response from the AI.
    """

    # Load reader model
    model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME)
    tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
    READER_LLM = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        do_sample=True,
        temperature=0.2,
        repetition_penalty=1.1,
        return_full_text=False,
        max_new_tokens=500,
    )

    # Build prompt
    prompt_in_chat_format_v2 = [
    {
        "role": "system",
        "content": """Use only the information contained in the provided context to generate a precise and relevant answer to the given question.  
        
        **General Rules:**  
        - Answer concisely and directly to the question.  
        - If the answer requires choosing from multiple options, explicitly state the correct answer.  
        - Always cite the sources used and explain their relevance to the answer.  
        - If the answer cannot be deduced from the context, explicitly state that it cannot be answered.  

        **For Multiple Choice Questions (MCQs):**  
        - Start your response with: **"The correct answer is: [option]"** (e.g., "The correct answer is: A.")  
        - Explain why this option is correct based on the provided context.  
        - Briefly justify why the other options are incorrect, if possible.  
        
        **Example of response format:**  
        - **The correct answer is: [option]**  
        - **Justification:** (Explain why this answer is correct, citing sources)  
        - **Why other options are incorrect:** (Briefly explain why the other options do not apply)  

        If the question is not a multiple-choice question, provide a direct and structured answer.  
        """,
    },
    {
        "role": "user",
        "content": """Context:  
    {context}  
    ---  
    Now, answer the following question.  

    **Question:** {question}""",
    }
    ]

    RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(prompt_in_chat_format_v2, tokenize=False, add_generation_prompt=True)
    

    # Retrieve context
    retrieved_docs = get_context(question)
    context = "\nExtracted documents:\n"
    context += "".join([f'Content: {doc.page_content} \nSource: {doc.metadata['ref']}\n\n' for i, doc in enumerate(retrieved_docs)])
    
    # Add context to prompt
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)
    
    # Redact an answer
    answer = READER_LLM(final_prompt)[0]["generated_text"]

    return answer, context

In [30]:
# Test function
question = """Your Client, A Inc, is a sub-licensee under European patent application EP-1. Can the sub-licence be recorded in the European Patent Register?
 
A    No, it is not possible to record sub-licences in the European Patent Register.
 
B    Yes, any sub-licence can be recorded in the European Patent Register.
 
C    Yes, provided the licensee granting the sub-licence has recorded its licence in the European Patent Register.
"""

answer, context = get_ai_answer(question)
print(f'--------------------- Question ---------------------\n\
    {question}\n\n\
    --------------------- Answer ---------------------\n\
    {answer}\n\nContext:{context}')

Device set to use cpu



Starting retrieval for query='Your Client, A Inc, is a sub-licensee under European patent application EP-1. Can the sub-licence be recorded in the European Patent Register?\n\nA    No, it is not possible to record sub-licences in the European Patent Register.\n\nB    Yes, any sub-licence can be recorded in the European Patent Register.\n\nC    Yes, provided the licensee granting the sub-licence has recorded its licence in the European Patent Register.\n'...
--------------------- Question ---------------------
    Your Client, A Inc, is a sub-licensee under European patent application EP-1. Can the sub-licence be recorded in the European Patent Register?

A    No, it is not possible to record sub-licences in the European Patent Register.

B    Yes, any sub-licence can be recorded in the European Patent Register.

C    Yes, provided the licensee granting the sub-licence has recorded its licence in the European Patent Register.


    --------------------- Answer ---------------------
   

### Tests *get_ai_answer()*

In [ ]:
# def get_ai_answer(question):
#     """
#     Fonction fictive qui génère une réponse IA et un contexte.
#     """
#     return "Réponse IA simulée", "Contexte IA simulé"

def get_ai_answer_evaluation(test_name, options={}):
    output_dir = "../outputs/"
    tests = []
    
    for filename in os.listdir(output_dir):
        if "solution" in filename or "json" not in filename or "MOCK" in filename: # on ignore MOCK
            continue  # On ignore les fichiers contenant "solution"
        
        filepath = os.path.join(output_dir, filename)
        print(filepath)
        with open(filepath, 'r', encoding='utf-8') as file:
            doc = json.load(file)
        
        questions = []
        answers = []
        
        if "mcq" in filename:
            for elt in doc.values():
                questions.append(f'{elt["question"]}\n {"\n".join(elt["options"])}')
            
            solution_filepath = filepath.rsplit('.json', 1)[0] + "_solution.json"

            if os.path.exists(solution_filepath):
                with open(solution_filepath, 'r', encoding='utf-8') as file:
                    solution_doc = json.load(file)
                for elt in solution_doc.values():
                    answers.append(f'Answer: {elt["Answer"]}\n\nJustification: {elt["Justification"]}')
            
            for i in range(len(questions)):
                answer, context = get_ai_answer(questions[i])
                AIAnswer = f'Answer: {answer}\nContext: {context}'
                tests.append({
                    'Nom doc': filename,
                    'Type': 'MCQ',
                    'Question': questions[i],
                    'Answer': answers[i], 
                    'AIAnswer': AIAnswer
                })

        
        elif "open" in filename:
            for elt in doc.values():
                questions.append(elt)
            
            solution_filepath = filepath.rsplit('open.json', 1)[0] + "solution_open.json"
            if os.path.exists(solution_filepath):
                with open(solution_filepath, 'r', encoding='utf-8') as file:
                    solution_doc = json.load(file)
                for elt in solution_doc.values():
                    answers.append(elt)
            
            for i in range(len(questions)):
                answer, context = get_ai_answer(questions[i])
                AIAnswer = f'Answer: {answer}\nContext: {context}'
                tests.append({
                    'Nom doc': filename,
                    'Type': 'Open',
                    'Question': questions[i],
                    'Answer': answers[i] if i < len(answers) else "", 
                    'AIAnswer': AIAnswer
                })
    
    final = {'options': options ,'tests': tests}

    folder_path = '../outputs/ai_answers_evaluation'
    # Vérifier si le dossier existe, sinon le créer
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    output_path = os.path.join(folder_path, f"{test_name}.json")
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(final, file, indent=4, ensure_ascii=False)
    
    return final

In [ ]:
READER_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
name = 'base_model'
options = {
    'embedding_model': 'thenlper/gte-small',
    'chunck_limit': 512,
    'reader_model': READER_MODEL_NAME,
    'Reranker': False,
    'Constrained output': False,
    'Agentic Rag': False,
}
get_ai_answer_evaluation(name, options)

../outputs/2024_EPAC_mcq.json
../outputs/2022_EPAC_mcq.json
../outputs/2022_EPAC_open.json
../outputs/ai_answers_evaluation_base_model.json
../outputs/2023_EPAC_mcq.json
../outputs/2023_EPAC_open.json
../outputs/2024_EPAC_open.json
../outputs/categories.json


{'options': {'embedding_model': 'thenlper/gte-small',
  'chunck_limit': 512,
  'reader_model': 'Qwen/Qwen2.5-1.5B-Instruct',
  'Reranker': False,
  'Constrained output': False,
  'Agentic Rag': False},
 'tests': [{'Nom doc': '2024_EPAC_mcq.json',
   'Type': 'MCQ',
   'Question': 'An international application is published , together  with the search report drawn up by the CNIPA , 18 m onths + 1 day after the filing date (no priority claimed). The application has no more than 35 pages and 15 claims. Which statement reflects all actions the CN  applicant  needs to take for entry into the EP phase 25 months  after the filing date ? A request for early processing has been filed.\n A. Complete and file Form 1200 and pay the filing fee and the search fee\nB. Complete and file Form 1200 and pay the filing fee, the search fee and the renewal fee for the third year\nC. Complete and file Form 1200, pay the filing fee and the search fee and appoint a representative\nD. None of the above statements

# Not implemented yet

## *get_question(subcategory)*
```
def get_question(sub_categorie)
	Récupère dans la DB de MAGB 3 questions au hasard de la bonne catégorie.
	Récupère du contexte pour ces 3 questions.
	Donne tout à un LLM qui génère une question.
	Lorsqu’on génère une question on génère sa réponse par IA en suivant, le temps que l’user réponse, pour enregistrer ça en BDD et donner rapidement un feedback à l’user.
	Dans la v2 un agent vérifie la qualité des sources et corrige.
```

## *get_feedback(question, reponse_ia, reponse_user)*

```
def get_feedback(question, reponse_ia, reponse_user)
	On récupère du contexte pour toutes ces entrés.
	On génère avec notre LLM un feed_back expliquant pk l’user s’est trompé, qu’est ce qui était mauvais.
	On lui renvoie la vraie réponse de l’IA en plus de notre analyse.
```

## *free_speach(history)*

```
def free_speach(history)
	Là c’est la fonctionnalité ou l’user peut discuter avec le modèle après avoir reçu un feedback. Il prend en entrée tout l’historique de la conv, c’est à dire question, réponse user, réponse et analyse ia. 
	Lorsqu’un user lui envoie un message il récupère du contexte dans le RAG et lui répond.
```